# Indicateurs QualiCharge : Structure

Ce Notebook présente une proposition de structuration et de représentation des indicateurs Qualicharge.

La liste des indicateurs est présentée sur [ce lien](https://loco-philippe.github.io/IRVE/files/indicateurs.html).

*Nota : La représentation visuelle des indicateurs (ex. diagramme circulaire, diagramme à barre, carte choroplèthe, courbe...) n'est pas abordée.*

In [1]:
import os
import json
from sqlalchemy import create_engine
import pandas as pd
from util import to_indicator

# Connecteur à la base Qualicharge
engine = create_engine(os.getenv("DATABASE_URL"))

## Structure des indicateurs

### Codification des indicateurs

Les indicateurs sont codifiés par une chaine de caractères *[type]-[périmètre]-[valeur de périmètre]-[critère]* avec :

- *type* : identifiant du type d'indicateur (ex. 'i1' : nombre de points de recharge)
- *périmètre et valeur de périmètre*: sous ensemble des données sur lequel appliquer l'indicateur. Les périmètres actuellement définis sont les suivants :
  - 00: national (sans valeur)
  - 01: région (valeur : code de la région)
  - 02: département (valeur : code du département)
  - 03: EPCI (valeur : code de l'EPCI)
  - 04: commune (valeur : code de la commune)
- *critère* : paramètre spécifique du type d'indicateur

Le périmètre par défaut est l'ensemble des données.


Exemples de codification :
- **t4-04-74012** : Pourcentage de stations par nombre de points de recharge (t4) pour la ville (04) d'Annemasse (74012)
- **i1-01-93** : Nombre de points de recharge (i1) pour la région (01) PACA (93)
- **i1-01-93-03** : Nombre de points de recharge (i1) pour la région (01) PACA (93) par EPCI (03)
- **t1** : Nombre de points de recharge par niveau de puissance (t1) pour l'ensemble des données (pas de périmètre choisi)


### Exemples de mise en oeuvre

'i1' est l'indicateur qui fournit le nombre de points de recharge.

Les colonnes de gauche sont les valeurs calculées liées à l'indicateur (ici 'nb_pdc').

Les colonnes de droites sont des données complémentaires:

- 'level' indique le type de périmètre et 'code' indique la valeur pour ce périmètre,
- 'name' est une information optionnelle décrivant le 'code'

*Nota : L'appartenance à une zone géographique se fait par le test d'appartenance d'un point à un polygone (impact sur le temps de calcul de certains indicateurs).*

In [2]:
# calcul sur l'ensemble des données ('i1' est équivalent à 'i1-00-00-00')
to_indicator(engine, 'i1')

,nb_pdc,level,code,name
0,13400,00,00,national


In [3]:
# calcul sur l'ensemble des données avec une répartition par région (01) ('i1---01' est équivalent à 'i1-00-00-01')
# ex. ligne 1 : 'level' 01 indique un périmètre région, 'code' 84 indique le code de la région.
to_indicator(engine, 'i1---01')[:5]

,nb_pdc,level,code,name
0,3538,01,84,Auvergne-Rhône-Alpes
1,2351,01,93,Provence-Alpes-Côte d'Azur
2,1391,01,11,Île-de-France
3,1082,01,44,Grand Est
4,1027,01,75,Nouvelle-Aquitaine


In [4]:
# calcul sur l'ensemble de la région (01) PACA (93) sans répartition ('i1-01-93' est équivalent à 'i1-01-93-00')
to_indicator(engine, 'i1-01-93')

,nb_pdc,level,code
0,2351,01,93


In [5]:
# calcul sur l'ensemble de la région (01) PACA (93) par département (02)
to_indicator(engine, 'i1-01-93-02')

,nb_pdc,level,code,name
0,782,02,83,Var
1,668,02,06,Alpes-Maritimes
2,268,02,13,Bouches-du-Rhône
3,262,02,05,Hautes-Alpes
4,214,02,84,Vaucluse
5,157,02,04,Alpes-de-Haute-Provence


### Options de représentation

La représentation par défaut est sous la forme d'un DataFrame pandas (voir exemples ci-dessus). 

Les représentations complémentaires sont présentées ci-dessous.

In [6]:
# Représentation sans les colonnes optionnelles (ici 'name')
to_indicator(engine, 'i1-01-93-02', simple=True)

,nb_pdc,level,code
0,782,02,83
1,668,02,06
2,268,02,13
3,262,02,05
4,214,02,84
5,157,02,04


Pour un indicateur donné (ex. 'i1'), la structure 'simple' est identique. L'historisation des données peut donc s'effectuer par indicateur (avec l'ajout d'un timestamp). 

Par exemple, on pourrait avoir une table 'i1-histo' et y stocker la valeur totale 'i1' quotidiennement et la valeur par région 'i1---01' mensuellement.

Pour avoir un historique sur le nombre de pdc en PACA, la table serait filtrée avec level=01 et code=93

In [7]:
# représentation avec un timestamp.
to_indicator(engine, 'i1-01-93-02', histo=True)

,nb_pdc,level,code,timest
0,782,02,83,2024-08-24 14:42:02.518384+00:00
1,668,02,06,2024-08-24 14:42:02.518384+00:00
2,268,02,13,2024-08-24 14:42:02.518384+00:00
3,262,02,05,2024-08-24 14:42:02.518384+00:00
4,214,02,84,2024-08-24 14:42:02.518384+00:00
5,157,02,04,2024-08-24 14:42:02.518384+00:00


In [8]:
# représentation sous forme de JSON
to_indicator(engine, 'i1-01-93-02', simple=True, format='json', json_orient='records')

'{"i1-01-93-02": [{"nb_pdc":782,"level":"02","code":"83"},{"nb_pdc":668,"level":"02","code":"06"},{"nb_pdc":268,"level":"02","code":"13"},{"nb_pdc":262,"level":"02","code":"05"},{"nb_pdc":214,"level":"02","code":"84"},{"nb_pdc":157,"level":"02","code":"04"}]}'

In [9]:
# représentation sous forme de table
to_indicator(engine, 'i1-01-93-02', format='table')

,count
0,6


In [10]:
# représentation sous forme de requète PostgreSQL
to_indicator(engine, 'i1', format='query')

" WITH national(code, name) AS (VALUES ('00', 'national')) , perimeter(level) AS (VALUES ('00'))  SELECT count(id_pdc_itinerance) AS nb_pdc, level, code, name  FROM perimeter, pointdecharge, national GROUP BY level, code, name "

## Infrastructure - quantitatif

Indicateurs pris en compte : 'i1', 'i4', 'i7'

Les autres indicateurs sont dérivés ('i2', 'i5', 'i8' ramené à 100 000 habitants et 'i3', 'i6', 'i9' ramené à 100 km2).

*à préciser : Quelle population retenir (date fixe ?) ? Est-ce qu'on stocke en base la surface (à partir des polygones) ?*

### I1 : Nombre de points de recharge ouverts au public

'nb_pdc' est le nombre de points de recharge.

In [11]:
to_indicator(engine, 'i1')

,nb_pdc,level,code,name
0,13400,00,00,national


In [12]:
to_indicator(engine, 'i1-00-00-00', simple=True)

,nb_pdc,level,code
0,13400,00,00


In [13]:
i1_nat = to_indicator(engine, 'i1-00-00-01')
print(i1_nat['nb_pdc'].sum())
i1_nat[:10]

13400


,nb_pdc,level,code,name
0,3538,01,84,Auvergne-Rhône-Alpes
1,2351,01,93,Provence-Alpes-Côte d'Azur
2,1391,01,11,Île-de-France
3,1082,01,44,Grand Est
4,1027,01,75,Nouvelle-Aquitaine
5,1021,01,76,Occitanie
6,732,01,32,Hauts-de-France
7,572,01,52,Pays de la Loire
8,565,01,27,Bourgogne-Franche-Comté
9,350,01,53,Bretagne


In [14]:
to_indicator(engine, 'i1-01-93-00')

,nb_pdc,level,code
0,2351,01,93


In [15]:
to_indicator(engine, 'i1-01-93')

,nb_pdc,level,code
0,2351,01,93


In [16]:
to_indicator(engine, 'i1-01')

,nb_pdc,level,code


In [17]:
paca_epci = 'i1-01-93-03'
i1_paca = to_indicator(engine, paca_epci)
i1_paca[:10]

,nb_pdc,level,code,name
0,264,03,248300543,Métropole Toulon-Provence-Méditerranée
1,258,03,200054807,Métropole d'Aix-Marseille-Provence
2,150,03,240600585,CA de Sophia Antipolis
3,145,03,200039857,CA du Pays de Grasse
4,136,03,200039915,CA Cannes Pays de Lérins
5,135,03,200035319,CA Estérel Côte d'Azur Agglomération
6,133,03,248400251,CA du Grand Avignon (COGA)
7,108,03,200030195,Métropole Nice Côte d'Azur
8,102,03,200036077,CC du Golfe de Saint-Tropez
9,69,03,200067825,CA Gap-Tallard-Durance


### I4 : Nombre de stations ouvertes au public

'nb_stat' est le nombre de stations.

In [18]:
to_indicator(engine, 'i4-0', simple=True)

,nb_stat,level,code
0,6062,00,00


In [19]:
i4_nat = to_indicator(engine, 'i4-0-xx-01')
print(i4_nat['nb_stat'].sum())
i4_nat[:10]

6062


,nb_stat,level,code,name
0,1722,01,84,Auvergne-Rhône-Alpes
1,1448,01,93,Provence-Alpes-Côte d'Azur
2,495,01,11,Île-de-France
3,424,01,44,Grand Est
4,355,01,75,Nouvelle-Aquitaine
5,310,01,76,Occitanie
6,274,01,52,Pays de la Loire
7,251,01,32,Hauts-de-France
8,211,01,27,Bourgogne-Franche-Comté
9,193,01,53,Bretagne


In [20]:
to_indicator(engine, 'i4-01-93-0')

,nb_stat,level,code
0,1448,01,93


In [21]:
to_indicator(engine, 'i4-01-93-03')[:10]

,nb_stat,level,code,name
0,185,03,248300543,Métropole Toulon-Provence-Méditerranée
1,143,03,240600585,CA de Sophia Antipolis
2,141,03,200039857,CA du Pays de Grasse
3,132,03,200039915,CA Cannes Pays de Lérins
4,103,03,200035319,CA Estérel Côte d'Azur Agglomération
5,100,03,200054807,Métropole d'Aix-Marseille-Provence
6,66,03,200039931,CC Alpes d'Azur
7,45,03,200036077,CC du Golfe de Saint-Tropez
8,44,03,200068104,CA de la Provence Verte
9,41,03,248400251,CA du Grand Avignon (COGA)


### I7 : Puissance installée

'p_nom' est la puissance nominale cumulée

In [22]:
to_indicator(engine, 'i7', simple=True)

,p_nom,level,code
0,907015.04,00,00


In [23]:
i7_nat = to_indicator(engine, 'i7-0--01')
print(i7_nat['p_nom'].sum())
i7_nat[:10]

907015.0399999991


,p_nom,level,code,name
0,171240.96,01,84,Auvergne-Rhône-Alpes
1,130036.72,01,11,Île-de-France
2,113339.94,01,76,Occitanie
3,89509.30,01,75,Nouvelle-Aquitaine
4,83123.12,01,93,Provence-Alpes-Côte d'Azur
5,80712.50,01,44,Grand Est
6,68916.88,01,32,Hauts-de-France
7,47298.44,01,27,Bourgogne-Franche-Comté
8,40000.24,01,52,Pays de la Loire
9,31518.80,01,28,Normandie


In [24]:
i7_paca_city = to_indicator(engine, 'i7-01-93-04', simple=True)
i7_paca_city[:10]

,p_nom,level,code
0,5334.40,04,06088
1,4222.00,04,13117
2,3956.00,04,13001
3,3642.40,04,84007
4,3204.28,04,13055
5,2400.00,04,13088
6,2354.00,04,13015
7,2332.00,04,84129
8,2166.00,04,05061
9,2154.24,04,83061


## Infrastructure - typologie

Les indicateurs 't1' à 't6' sont pris en compte.

L'indicateur 't7' reste à construire (non prioritaire).

Les autres indicateurs sont à définir (

### T1 : Nombre de points de recharge par niveau de puissance

'nb_pdc' est le nombre de points de recharge.

'p-range' est la plage de puissance (ex. [65, 175) -> de 65 inclus à 175 exclus)

'p-cat' est l'index de la catégorie (1 pour la catégorie la plus basse)

In [25]:
t1_nat = to_indicator(engine, 't1-00')
print(t1_nat['nb_pdc'].sum())
t1_nat

13400


,nb_pdc,p_cat,p_range,level,code,name
0,7602,2,"[15.0, 26.0)",00,00,national
1,1928,4,"[65, 175.0)",00,00,national
2,1344,5,"[175, 360.0)",00,00,national
3,1310,1,"[0, 15.0)",00,00,national
4,1129,3,"[26, 65.0)",00,00,national
5,87,6,"[360, None)",00,00,national


In [26]:
to_indicator(engine, 't1', simple=True)

,nb_pdc,p_cat,p_range,level,code
0,7602,2,"[15.0, 26.0)",00,00
1,1928,4,"[65, 175.0)",00,00
2,1344,5,"[175, 360.0)",00,00
3,1310,1,"[0, 15.0)",00,00
4,1129,3,"[26, 65.0)",00,00
5,87,6,"[360, None)",00,00


In [27]:
to_indicator(engine, 't1-02-75')

,nb_pdc,p_cat,p_range,level,code,name
0,59,2,"[15.0, 26.0)",02,75,Paris
1,12,1,"[0, 15.0)",02,75,Paris
2,10,4,"[65, 175.0)",02,75,Paris
3,3,3,"[26, 65.0)",02,75,Paris
4,2,5,"[175, 360.0)",02,75,Paris


In [28]:
to_indicator(engine, 't1-02')

,nb_pdc,p_cat,p_range,level,code,name


### T2 : Pourcentage de points de recharge par niveau de puissance

Indicateur similaire à 't1' ( 'pct_nb_pdc' remplace 'nb_pdc').

'pct_nb_pdc' est le pourcentage de pdc pour le niveau de puissance.

In [29]:
to_indicator(engine, 't2')

,pct_nb_pdc,p_cat,p_range,level,code,name
0,56.731343,2,"[15.0, 26.0)",00,00,national
1,14.388060,4,"[65, 175.0)",00,00,national
2,10.029851,5,"[175, 360.0)",00,00,national
3,9.776119,1,"[0, 15.0)",00,00,national
4,8.425373,3,"[26, 65.0)",00,00,national
5,0.649254,6,"[360, None)",00,00,national


In [30]:
to_indicator(engine, 't2-02-75', simple=True)

,pct_nb_pdc,p_cat,p_range,level,code
0,68.604651,2,"[15.0, 26.0)",02,75
1,13.953488,1,"[0, 15.0)",02,75
2,11.627907,4,"[65, 175.0)",02,75
3,3.488372,3,"[26, 65.0)",02,75
4,2.325581,5,"[175, 360.0)",02,75


### T3 : Nombre de stations par nombre de points de recharge

'nb_stations' est le nombre de stations.

'nb_pdc' est le nombre de pdc.

ex. il y a 2790 stations (nb_stations) avec un seul pdc (nb_pdc).

In [31]:
to_indicator(engine, 't3-00')[:10]

,nb_stations,nb_pdc,level,code,name
0,2790,1,00,00,national
1,2142,2,00,00,national
2,356,4,00,00,national
3,256,3,00,00,national
4,165,6,00,00,national
5,155,5,00,00,national
6,55,8,00,00,national
7,25,9,00,00,national
8,25,7,00,00,national
9,20,12,00,00,national


In [32]:
to_indicator(engine, 't3-04-74012')

,nb_stations,nb_pdc,level,code,name
0,6,1,04,74012,Annemasse
1,3,2,04,74012,Annemasse
2,1,4,04,74012,Annemasse


### T4 : Pourcentage de stations par nombre de points de recharge

Indicateur similaire à 't3' ( 'pct_nb_stations' remplace 'nb_stations').

'pct_nb_stations' est le pourcentage de stations avec un nombre de pdc donné.

In [33]:
to_indicator(engine, 't4')[:10]

,pct_nb_stations,nb_pdc,level,code,name
0,46.024414,1,00,00,national
1,35.334873,2,00,00,national
2,5.872649,4,00,00,national
3,4.223029,3,00,00,national
4,2.721874,6,00,00,national
5,2.556912,5,00,00,national
6,0.907291,8,00,00,national
7,0.412405,9,00,00,national
8,0.412405,7,00,00,national
9,0.329924,12,00,00,national


In [34]:
to_indicator(engine, 't4-04-74012', simple=True)

,pct_nb_stations,nb_pdc,level,code
0,60.0,1,04,74012
1,30.0,2,04,74012
2,10.0,4,04,74012


### T5 : Nombre de stations par type d’implantation

'nb_stations' est le nombre de stations.

'implantation' est le type d'implantation

In [35]:
t5_nat = to_indicator(engine, 't5-00')
print(t5_nat['nb_stations'].sum())
t5_nat[:10]

6062


,nb_stations,implantation,level,code,name
0,3219,VOIRIE,00,00,national
1,1328,PARKING_PRIVE_USAGE_PUBLIC,00,00,national
2,965,PARKING_PUBLIC,00,00,national
3,535,STATION_RECHARGE_RAPIDE,00,00,national
4,15,PARKING_PRIVE_CLIENTELE,00,00,national


In [36]:
to_indicator(engine, 't5', simple=True)

,nb_stations,implantation,level,code
0,3219,VOIRIE,00,00
1,1328,PARKING_PRIVE_USAGE_PUBLIC,00,00
2,965,PARKING_PUBLIC,00,00
3,535,STATION_RECHARGE_RAPIDE,00,00
4,15,PARKING_PRIVE_CLIENTELE,00,00


In [37]:
to_indicator(engine, 't5-03-200023414')

,nb_stations,implantation,level,code,name
0,14,VOIRIE,03,200023414,Métropole Rouen Normandie
1,10,PARKING_PRIVE_USAGE_PUBLIC,03,200023414,Métropole Rouen Normandie
2,7,STATION_RECHARGE_RAPIDE,03,200023414,Métropole Rouen Normandie
3,1,PARKING_PUBLIC,03,200023414,Métropole Rouen Normandie


In [38]:
to_indicator(engine, 't5-03-200023414', simple=True)

,nb_stations,implantation,level,code
0,14,VOIRIE,03,200023414
1,10,PARKING_PRIVE_USAGE_PUBLIC,03,200023414
2,7,STATION_RECHARGE_RAPIDE,03,200023414
3,1,PARKING_PUBLIC,03,200023414


### T6 : Pourcentage de stations par type d’implantation

Indicateur similaire à 't5' ( 'pct_nb_stations' remplace 'nb_stations').

'pct_nb_stations' est le pourcentage de stations avec un type d'implantation donné.

In [39]:
to_indicator(engine, 't6')

,pct_nb_stations,implantation,level,code,name
0,53.101287,VOIRIE,00,00,national
1,21.906961,PARKING_PRIVE_USAGE_PUBLIC,00,00,national
2,15.918839,PARKING_PUBLIC,00,00,national
3,8.825470,STATION_RECHARGE_RAPIDE,00,00,national
4,0.247443,PARKING_PRIVE_CLIENTELE,00,00,national


In [40]:
to_indicator(engine, 't6-03-200023414')

,pct_nb_stations,implantation,level,code,name
0,43.750,VOIRIE,03,200023414,Métropole Rouen Normandie
1,31.250,PARKING_PRIVE_USAGE_PUBLIC,03,200023414,Métropole Rouen Normandie
2,21.875,STATION_RECHARGE_RAPIDE,03,200023414,Métropole Rouen Normandie
3,3.125,PARKING_PUBLIC,03,200023414,Métropole Rouen Normandie


In [41]:
to_indicator(engine, 't6-03-200023414', simple=True)

,pct_nb_stations,implantation,level,code
0,43.750,VOIRIE,03,200023414
1,31.250,PARKING_PRIVE_USAGE_PUBLIC,03,200023414
2,21.875,STATION_RECHARGE_RAPIDE,03,200023414
3,3.125,PARKING_PUBLIC,03,200023414


### Autres indicateurs de typologie

Les indicateurs liés à d'autres typologies (ex. opérateurs, accès deux roues, période d’ouverture, accès handicapés…) sont à définir.

Ceux concernant les opérateurs sont prioritaires.

## Autres indicateurs à prendre en compte

- Indicateurs d'historique (traitement des données historisées)
- Infrastructure - réseau autoroute (nécessite l'identification des stations de ce réseau)
- Usage - quantitatif (traitement des données dynamiques) 
- Usage - qualité de service (traitement des données dynamiques)
- Indicateurs étendus (en lien avec des données externes - ex. trafic, immatriculation, consommation ENEDIS)